In [1]:
import sys
sys.path.append("..")
import numpy as np
from project.ship import Ship
from project.tests import Tests
from project.botma import TMA
import time

In [2]:
# Класс для сохранения результатов
tests = Tests('test1')

# Создаем наблюдатель
observer_x, observer_y, observer_course, observer_velocity = 0, 0, 0, 5
observer = Ship('Наблюдатель', observer_x, observer_y, observer_course,
                observer_velocity, verbose=True)

In [3]:
# Моделирование траектории наблюдателя
observer.forward_movement(3 * 60)
observer.change_course(270, 'left', omega=0.5)
observer.forward_movement(5 * 60)
observer.change_course(90, 'right', omega=0.5)
observer.forward_movement(3 * 60)

Наблюдатель движется прямо по курсу 0.0° 180с
Наблюдатель перешёл на курс 270.0° за 180с
Наблюдатель движется прямо по курсу 270.0° 300с
Наблюдатель перешёл на курс 90.0° за 360с
Наблюдатель движется прямо по курсу 90.0° 180с


In [4]:
def target_func(seed=None):
    np.random.seed(seed)
    b = 0
    d = 10
    c = np.random.uniform(0, 180)
    v = np.random.uniform(3, 15)
    return [b, d, c, v]
p0 = [0., 25., 90., 7.]
std_arr = [0., 0.1, 0.2, 0.3, 0.5, 1.]
precisions_1 = ['{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.3f}', '{0:.2f}',
'{0:.2f}','{0:.3f}','{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{0:.2f}', '{:.0f}', '{0:.4f}', '{0:.0f}', '{0:.0f}']
precisions_2 = ['{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.1f}', '{0:.3f}', '{0:.1f}', '{0:.2f}', '{0:.2f}','{0:.3f}', '{0:.2f}', '{0:.2f}', '{0:.0f}', '{0:.4f}']

In [5]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2)
    tma.print_verbose()
    print('ММП v2')
    start_time = time.perf_counter()
    dict_results = tma.swarm(n=10, target_func=target_func, p0=p0, seeded=True)
    stop_time = time.perf_counter()
    print('Моделирование 10 результатов закончено за t = {:.1f} с'.format(stop_time - start_time))
    df = tests.get_df(dict_results)

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['СКО X', 'СКО Y', 'СКО VX', 'СКО VY', 'Кб', 'П0_апр', 'Д0_апр', 'К0_апр', 'V0_апр'], axis=1))

    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}$, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('Nf = {:.0f}, Nf_max = {:.0f}'.format(df['Nf'].mean(), max(df['Nf'])))
    print('Iter = {:.0f}, Iter_max = {:.0f}'.format(df['Iter'].mean(), max(df['Iter'])))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))

    tests.save_df(df, name='../tests/tests8/ЛМ ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_______________________________________________')

СКОп = 0.0, tau = 2
ММП v2
Моделирование 10 результатов закончено за t = 0.1 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,-0.0,10.000,11.446,-1.77,0.34,15.807,0.00,"[1, 1, 1, 1]",0.0053,19,15
1,0.0,10.000,75.064,11.64,0.0,10.000,11.250,3.00,0.64,18.328,0.00,"[1, 1, 1, 1]",0.0025,12,10
2,0.0,10.000,78.479,3.31,-0.0,10.000,3.244,0.66,1.02,9.618,0.00,"[1, 1, 1, 1]",0.0018,8,7
3,0.0,10.000,99.144,11.50,-0.0,10.000,11.352,-1.83,0.34,15.677,0.00,"[1, 1, 1, 1]",0.0039,19,15
4,0.0,10.000,174.065,9.57,0.0,10.000,0.989,-9.52,-1.04,4.673,0.00,"[1, 1, 1, 1]",0.0057,24,18
5,0.0,10.000,39.959,13.45,-0.0,10.000,8.637,10.31,1.04,22.875,0.00,"[1, 1, 1, 1]",0.0019,9,8
6,0.0,10.000,160.715,6.98,-0.0,10.000,2.307,-6.59,-0.13,3.973,0.00,"[1, 1, 1, 1]",0.0049,27,21
7,0.0,10.000,13.735,12.36,-0.0,10.000,2.935,12.01,1.36,22.290,0.00,"[1, 1, 1, 1]",0.0016,8,7
8,0.0,10.000,157.217,14.62,-0.0,10.000,5.662,-13.48,-0.83,11.865,0.00,"[1, 1, 1, 1]",0.0045,23,17
9,0.0,10.000,1.867,9.02,0.0,10.000,0.294,9.02,1.49,18.268,0.00,"[1, 1, 1, 1]",0.0016,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.00
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0034, t_max = 0.0057
_______________________________________________
СКОп = 0.1, tau = 2
ММП v2
Моделирование 10 результатов закончено за t = 0.1 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,-0.0,10.026,11.496,-1.78,0.34,15.807,0.01,"[1, 1, 1, 1]",0.0039,19,15
1,0.0,10.000,75.064,11.64,0.0,9.965,11.193,2.99,0.64,18.328,0.01,"[1, 1, 1, 1]",0.0024,12,10
2,0.0,10.000,78.479,3.31,0.0,10.008,3.236,0.64,1.02,9.618,0.01,"[1, 1, 1, 1]",0.0018,8,7
3,0.0,10.000,99.144,11.50,0.0,9.982,11.310,-1.83,0.34,15.677,0.01,"[1, 1, 1, 1]",0.0037,19,15
4,0.0,10.000,174.065,9.57,-0.0,10.016,0.995,-9.53,-1.04,4.673,0.01,"[1, 1, 1, 1]",0.0047,24,18
5,0.0,10.000,39.959,13.45,-0.0,10.017,8.670,10.34,1.04,22.875,0.01,"[1, 1, 1, 1]",0.0019,9,8
6,0.0,10.000,160.715,6.98,-0.0,10.057,2.343,-6.64,-0.13,3.973,0.01,"[1, 1, 1, 1]",0.0050,27,21
7,0.0,10.000,13.735,12.36,0.0,9.961,2.907,11.94,1.36,22.290,0.01,"[1, 1, 1, 1]",0.0022,8,7
8,0.0,10.000,157.217,14.62,-0.0,9.989,5.656,-13.46,-0.83,11.865,0.01,"[1, 1, 1, 1]",0.0044,23,17
9,0.0,10.000,1.867,9.02,-0.0,10.024,0.295,9.00,1.49,18.268,0.01,"[1, 1, 1, 1]",0.0015,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.01
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0031, t_max = 0.0050
_______________________________________________
СКОп = 0.2, tau = 2
ММП v2
Моделирование 10 результатов закончено за t = 0.1 с


...

Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.09
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0031, t_max = 0.0050
_______________________________________________
СКОп = 0.5, tau = 2
ММП v2
Моделирование 10 результатов закончено за t = 0.1 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,-0.0,10.134,11.695,-1.83,0.34,15.807,0.25,"[1, 1, 1, 1]",0.0038,19,15
1,0.0,10.000,75.064,11.64,0.0,9.830,10.966,2.96,0.64,18.328,0.25,"[1, 1, 1, 1]",0.0024,12,10
2,0.0,10.000,78.479,3.31,0.0,10.037,3.202,0.56,1.02,9.618,0.26,"[1, 1, 1, 1]",0.0018,8,7
3,0.0,10.000,99.144,11.50,0.0,9.910,11.144,-1.82,0.34,15.677,0.26,"[1, 1, 1, 1]",0.0036,19,15
4,0.0,10.000,174.065,9.57,-0.0,10.080,1.019,-9.61,-1.04,4.673,0.23,"[1, 1, 1, 1]",0.0046,24,18
5,0.0,10.000,39.959,13.45,-0.0,10.087,8.800,10.46,1.04,22.875,0.24,"[1, 1, 1, 1]",0.0019,9,8
6,0.0,10.000,160.715,6.98,-0.0,10.290,2.491,-6.85,-0.13,3.973,0.23,"[0, 1, 1, 1]",0.0052,27,21
7,0.0,10.000,13.735,12.36,0.0,9.808,2.797,11.68,1.36,22.290,0.24,"[1, 1, 1, 1]",0.0016,8,7
8,0.0,10.000,157.217,14.62,-0.0,9.945,5.630,-13.39,-0.83,11.865,0.27,"[1, 1, 1, 1]",0.0044,23,17
9,0.0,10.000,1.867,9.02,-0.0,10.123,0.300,8.91,1.49,18.268,0.26,"[1, 1, 1, 1]",0.0015,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 90$, Рэф_мин = 0
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.25
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0031, t_max = 0.0052
_______________________________________________
СКОп = 1.0, tau = 2
ММП v2
Моделирование 10 результатов закончено за t = 0.1 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Ка,Точ,t,Nf,Iter
0,0.0,10.000,98.786,11.58,-0.0,10.271,11.951,-1.89,0.34,15.807,1.00,"[1, 1, 1, 1]",0.0038,19,15
1,0.0,10.000,75.064,11.64,0.0,9.668,10.692,2.93,0.64,18.328,0.99,"[1, 1, 1, 1]",0.0023,12,10
2,0.0,10.000,78.479,3.31,0.0,10.074,3.160,0.46,1.02,9.618,1.05,"[1, 1, 1, 1]",0.0019,8,7
3,0.0,10.000,99.144,11.50,0.0,9.821,10.938,-1.81,0.34,15.677,1.05,"[1, 1, 1, 1]",0.0037,19,15
4,0.0,10.000,174.065,9.57,-0.0,10.162,1.050,-9.71,-1.04,4.673,0.94,"[1, 1, 1, 1]",0.0045,24,18
5,0.0,10.000,39.959,13.45,-0.0,10.176,8.965,10.62,1.04,22.875,0.96,"[1, 1, 1, 1]",0.0022,9,8
6,0.0,10.000,160.715,6.98,-0.0,10.598,2.687,-7.13,-0.13,3.973,0.93,"[0, 0, 1, 1]",0.0051,27,21
7,0.0,10.000,13.735,12.36,0.0,9.627,2.666,11.37,1.36,22.290,0.95,"[0, 1, 1, 1]",0.0016,8,7
8,0.0,10.000,157.217,14.62,-0.0,9.890,5.598,-13.31,-0.83,11.865,1.09,"[1, 1, 1, 1]",0.0056,23,17
9,0.0,10.000,1.867,9.02,-0.0,10.246,0.306,8.79,1.49,18.268,1.05,"[1, 1, 1, 1]",0.0015,7,6


Показатели Монте-Карло:
Вариант 0: Рэф = 80$, Рэф_мин = 0
Вариант 1: Рэф = 90$, Рэф_мин = 0
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 1.00
Nf = 16, Nf_max = 27
Iter = 12, Iter_max = 21
t = 0.0032, t_max = 0.0056
_______________________________________________


In [6]:
for std in std_arr:

    tma = TMA(observer, std=np.radians(std), tau=2)
    tma.print_verbose()
    algorithm_name = 'Метод N пеленгов'
    print(algorithm_name)
    start_time = time.perf_counter()
    dict_results = tma.swarm(n=10, algorithm_name=algorithm_name, target_func=target_func, p0=p0, seeded=True)
    stop_time = time.perf_counter()
    print('Моделирование 10 результатов закончено за t = {:.1f} с'.format(stop_time - start_time))
    df = tests.get_df(dict_results).dropna(axis=1)

    d_df = df.copy()
    for i, j in enumerate(df.columns):
        try: d_df[j] = df[j].apply(lambda x: precisions_1[i].format(np.float64(x)))
        except(TypeError): pass
    display(d_df.drop(['Кб'], axis=1))

    print('Показатели Монте-Карло:')
    for i in range(4):
        a = df['Точ'].apply(lambda x: x[i])
        print('Вариант {}: Рэф = {:.0f}$, Рэф_мин = {:.0f}'.format(i, 100 * a.mean(), min(a)))
    print('Ка = {:.2f}'.format(df['Ка'].mean()))
    print('t = {:.4f}, t_max = {:.4f}'.format(df['t'].mean(), max(df['t'])))
    
    tests.save_df(df, name='../tests/tests8/N ' + str(np.degrees(tma.standart_deviation)) + '.xlsx')
    print('_______________________________________________________')

СКОп = 0.0, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.0 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Ка,Точ,t
0,0.0,10.000,98.786,11.58,0.0,10.000,11.446,-1.77,0.3,15.807,0.000,"[1, 1, 1, 1]",0.000
1,0.0,10.000,75.064,11.64,0.0,10.000,11.250,3.00,0.6,18.328,0.000,"[1, 1, 1, 1]",0.000
2,0.0,10.000,78.479,3.31,0.0,10.000,3.244,0.66,1.0,9.618,0.000,"[1, 1, 1, 1]",0.000
3,0.0,10.000,99.144,11.50,0.0,10.000,11.352,-1.83,0.3,15.677,0.000,"[1, 1, 1, 1]",0.000
4,0.0,10.000,174.065,9.57,0.0,10.000,0.989,-9.52,-1.0,4.673,0.000,"[1, 1, 1, 1]",0.000
5,0.0,10.000,39.959,13.45,0.0,10.000,8.637,10.31,1.0,22.875,0.000,"[1, 1, 1, 1]",0.000
6,0.0,10.000,160.715,6.98,0.0,10.000,2.307,-6.59,-0.1,3.973,0.000,"[1, 1, 1, 1]",0.000
7,0.0,10.000,13.735,12.36,0.0,10.000,2.935,12.01,1.4,22.290,0.000,"[1, 1, 1, 1]",0.000
8,0.0,10.000,157.217,14.62,0.0,10.000,5.662,-13.48,-0.8,11.865,0.000,"[1, 1, 1, 1]",0.000
9,0.0,10.000,1.867,9.02,0.0,10.000,0.294,9.02,1.5,18.268,0.000,"[1, 1, 1, 1]",0.000


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.00
t = 0.0003, t_max = 0.0003
_______________________________________________________
СКОп = 0.1, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.0 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Ка,Точ,t
0,0.0,10.000,98.786,11.58,-0.0,10.151,11.769,-1.80,0.3,15.807,0.012,"[1, 1, 1, 1]",0.000
1,0.0,10.000,75.064,11.64,-0.0,9.999,11.364,3.07,0.6,18.328,0.012,"[1, 1, 1, 1]",0.000
2,0.0,10.000,78.479,3.31,0.0,9.995,3.209,0.62,1.0,9.618,0.011,"[1, 1, 1, 1]",0.000
3,0.0,10.000,99.144,11.50,-0.0,10.118,11.627,-1.84,0.3,15.677,0.014,"[1, 1, 1, 1]",0.000
4,0.0,10.000,174.065,9.57,-0.0,9.999,0.990,-9.51,-1.0,4.673,0.009,"[1, 1, 1, 1]",0.000
5,0.0,10.000,39.959,13.45,-0.0,9.987,8.657,10.34,1.0,22.875,0.010,"[1, 1, 1, 1]",0.000
6,0.0,10.000,160.715,6.98,0.0,9.908,2.249,-6.51,-0.1,3.973,0.010,"[1, 1, 1, 1]",0.000
7,0.0,10.000,13.735,12.36,-0.0,9.774,2.973,12.27,1.4,22.290,0.012,"[1, 1, 1, 1]",0.000
8,0.0,10.000,157.217,14.62,-0.0,9.957,5.634,-13.41,-0.8,11.865,0.011,"[1, 1, 1, 1]",0.000
9,0.0,10.000,1.867,9.02,-0.0,10.020,0.289,8.96,1.5,18.268,0.010,"[1, 1, 1, 1]",0.000


Показатели Монте-Карло:
Вариант 0: Рэф = 100$, Рэф_мин = 1
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.01
t = 0.0002, t_max = 0.0004
_______________________________________________________
СКОп = 0.2, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.0 с


...

Показатели Монте-Карло:
Вариант 0: Рэф = 80$, Рэф_мин = 0
Вариант 1: Рэф = 100$, Рэф_мин = 1
Вариант 2: Рэф = 100$, Рэф_мин = 1
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.10
t = 0.0002, t_max = 0.0003
_______________________________________________________
СКОп = 0.5, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.0 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Ка,Точ,t
0,0.0,10.000,98.786,11.58,-0.1,9.109,10.606,-1.31,0.3,15.807,0.381,"[0, 1, 1, 1]",0.000
1,0.0,10.000,75.064,11.64,-0.1,9.002,10.282,3.15,0.6,18.328,0.360,"[0, 1, 1, 1]",0.000
2,0.0,10.000,78.479,3.31,0.0,9.733,2.905,0.44,1.0,9.618,0.275,"[0, 0, 0, 1]",0.000
3,0.0,10.000,99.144,11.50,-0.1,8.863,10.162,-1.24,0.3,15.677,0.457,"[0, 0, 0, 1]",0.000
4,0.0,10.000,174.065,9.57,-0.0,9.759,0.922,-9.22,-1.0,4.673,0.247,"[1, 1, 1, 1]",0.000
5,0.0,10.000,39.959,13.45,-0.0,9.282,7.878,9.70,1.0,22.875,0.269,"[0, 1, 1, 1]",0.000
6,0.0,10.000,160.715,6.98,0.0,9.160,1.835,-5.83,-0.1,3.973,0.250,"[0, 0, 0, 1]",0.000
7,0.0,10.000,13.735,12.36,-0.1,8.517,2.780,12.42,1.4,22.290,0.335,"[1, 1, 1, 1]",0.000
8,0.0,10.000,157.217,14.62,-0.0,9.350,5.187,-12.45,-0.8,11.865,0.302,"[0, 1, 1, 1]",0.000
9,0.0,10.000,1.867,9.02,-0.0,9.851,0.150,8.16,1.5,18.268,0.275,"[0, 1, 1, 1]",0.000


Показатели Монте-Карло:
Вариант 0: Рэф = 20$, Рэф_мин = 0
Вариант 1: Рэф = 70$, Рэф_мин = 0
Вариант 2: Рэф = 70$, Рэф_мин = 0
Вариант 3: Рэф = 100$, Рэф_мин = 1
Ка = 0.32
t = 0.0002, t_max = 0.0003
_______________________________________________________
СКОп = 1.0, tau = 2
Метод N пеленгов
Моделирование 10 результатов закончено за t = 0.0 с


,П0_ист,Д0_ист,К0_ист,V0_ист,П0_расч,Д0_расч,К0_расч,V0_расч,Птек_ист,Дтек_ист,Ка,Точ,t
0,0.0,10.000,98.786,11.58,-0.2,5.749,5.830,0.01,0.3,15.807,3.088,"[0, 0, 0, 0]",0.000
1,0.0,10.000,75.064,11.64,-0.2,6.609,6.978,2.90,0.6,18.328,2.170,"[0, 0, 0, 0]",0.000
2,0.0,10.000,78.479,3.31,0.1,8.991,2.259,0.22,1.0,9.618,1.218,"[0, 0, 0, 0]",0.000
3,0.0,10.000,99.144,11.50,-0.2,5.509,5.422,0.09,0.3,15.677,3.675,"[0, 0, 0, 0]",0.000
4,0.0,10.000,174.065,9.57,-0.0,9.014,0.704,-8.30,-1.0,4.673,1.122,"[0, 0, 0, 1]",0.000
5,0.0,10.000,39.959,13.45,-0.1,7.520,5.718,7.83,1.0,22.875,1.421,"[0, 0, 0, 0]",0.000
6,0.0,10.000,160.715,6.98,0.0,7.765,1.105,-4.57,-0.1,3.973,1.145,"[0, 0, 0, 0]",0.000
7,0.0,10.000,13.735,12.36,-0.2,6.650,2.038,11.16,1.4,22.290,1.641,"[0, 0, 0, 0]",0.000
8,0.0,10.000,157.217,14.62,-0.0,7.955,4.141,-10.23,-0.8,11.865,1.505,"[0, 0, 0, 0]",0.000
9,0.0,10.000,1.867,9.02,-0.0,9.174,-0.206,6.37,1.5,18.268,1.262,"[0, 0, 0, 0]",0.000


Показатели Монте-Карло:
Вариант 0: Рэф = 0$, Рэф_мин = 0
Вариант 1: Рэф = 0$, Рэф_мин = 0
Вариант 2: Рэф = 0$, Рэф_мин = 0
Вариант 3: Рэф = 10$, Рэф_мин = 0
Ка = 1.82
t = 0.0002, t_max = 0.0002
_______________________________________________________
